# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database


In [ ]:
# review the collections in our new database


In [10]:
# review a document in the establishments collection
document = establishments.find_one()
pprint(document)

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6686580417026598e76a133d'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_establishment = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [19]:
# Insert the new restaurant into the collection
establishment_update = establishments.insert_one(new_establishment)

DuplicateKeyError: E11000 duplicate key error collection: uk_food.establishments index: _id_ dup key: { _id: ObjectId('66865c103fcfddff4f4d038b') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: uk_food.establishments index: _id_ dup key: { _id: ObjectId('66865c103fcfddff4f4d038b') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('66865c103fcfddff4f4d038b')}}

In [20]:
# Check that the new restaurant was inserted
print(establisment_update.inserted_id)

NameError: name 'establisment_update' is not defined

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [25]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
establishments_collection = db['establishments']


query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

result = establishments_collection.find_one(query, projection)


print(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [31]:
# Update the new restaurant with the correct BusinessTypeID

projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

business_type = establishments_collection.find_one(projection)

if business_type:
    business_type_id = business_type['BusinessTypeID']
    
    new_restaurant_id = '1'
    
    update_query = {"_id": new_restaurant_id}
    new_values = {"$set": {"BusinessTypeID": business_type_id}}

    update_result = establishments_collection.update_one(update_query, new_values)


In [32]:
# Confirm that the new restaurant was updated
print(update_result.modified_count)

0


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [34]:
# Find how many documents have LocalAuthorityName as "Dover"

dover_query = {"LocalAuthorityName": "Dover"}

dover_count = establishments_collection.count_documents(dover_query)

print("Number of documents with Dover Local Authority", {dover_count})


Number of documents with Dover Local Authority {994}


In [37]:
# Delete all documents where LocalAuthorityName is "Dover"

delete_result = establishments_collection.delete_many(dover_query)


In [40]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments_collection.count_documents(dover_query)
print("Number of documents with Dover after delete", {remaining_dover_count})



Number of documents with Dover after delete {0}


In [41]:
# Check that other documents remain with 'find_one'
remaining_document = establishments_collection.find_one({"LocalAuthorityName": {"$ne": "Dover"}})
if remaining_document:
    print("Other documents remain in the collection:")
    print(remaining_document)
else:
    print("No other documents remain in the collection.")


Other documents remain in the collection:
{'_id': ObjectId('6686580417026598e76a1624'), 'FHRSID': 289352, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000002460', 'BusinessName': 'The Ship', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': '23 The Stade', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6AB', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2015-08-27T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.188537', 'latitude': '51.08084'}, 'RightToReply': '', 'Distance': 4591.912144538092, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'it

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [47]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal
from pymongo import MongoClient, UpdateOne

filter_query = {
    'geocode.longitude': {'$type': 'string'},
    'geocode.latitude': {'$type': 'string'}
}

update_operations = []

for document in establishments_collection.find(filter_query):
    longitude_str = document['geocode']['longitude']
    latitude_str = document['geocode']['latitude']
    
    longitude_decimal = Decimal(longitude_str)
    latitude_decimal = Decimal(latitude_str)
    
    update_operations.append(
        UpdateOne(
            {'_id': document['_id']},
            {'$set': {'geocode.longitude': longitude_decimal, 'geocode.latitude': latitude_decimal}}
        )
    )


Use `update_many` to convert `RatingValue` to integer numbers.

In [42]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [50]:
# Change the data type from String to Integer for RatingValue

filter_query = {
    'RatingValue': {'$type': 'string'}  
}

update_operations = []

for document in establishments_collection.find(filter_query):
    rating_value_str = document['RatingValue']
    try:
        rating_value_int = int(rating_value_str)
    except ValueError:
        rating_value_int = None  
    
    update_operations.append(
        UpdateOne(
            {'_id': document['_id']},
            {'$set': {'RatingValue': rating_value_int}}
        )
    )


In [49]:
# Check that the coordinates and rating value are now numbers
count_longitude_strings = establishments_collection.count_documents({
    'geocode.longitude': {'$type': 'string'}
})

count_latitude_strings = establishments_collection.count_documents({
    'geocode.latitude': {'$type': 'string'}
})

count_rating_strings = establishments_collection.count_documents({
    'RatingValue': {'$type': 'string'}
})

print(f"Documents with longitude as string: {count_longitude_strings}")
print(f"Documents with latitude as string: {count_latitude_strings}")
print(f"Documents with RatingValue as string: {count_rating_strings}")

Documents with longitude as string: 38786
Documents with latitude as string: 38786
Documents with RatingValue as string: 34694
